In [1]:
%%bash
cat << EOF > ./init_boundary/main.tf 
# https://developer.hashicorp.com/boundary/docs/install-boundary/initialize#create-your-first-login-account
provider "boundary" {
  addr             = "$(terraform output -raw fqdn_443)"
  tls_insecure     = true
  recovery_kms_hcl = <<EOT
    kms "gcpckms" {
    purpose     = "recovery"
    key_ring    = "$(terraform output -raw key_ring)"
    crypto_key  = "$(terraform output -raw crypto_key)"
    project     = "$(terraform output -raw project)"
    region      = "$(terraform output -raw location)"
    }
EOT
}

resource "boundary_scope" "org" {
  scope_id    = "global"
  name        = "global"
  description = "Organization scope"

  auto_create_admin_role   = false
  auto_create_default_role = false
}

resource "boundary_scope" "project" {
  name                     = "project"
  description              = "My first project"
  scope_id                 = boundary_scope.org.id
  auto_create_admin_role   = false
  auto_create_default_role = false
}

resource "boundary_auth_method" "password" {
  name        = "Password auth method"
  description = "Password auth method"
  type        = "password"
  scope_id    = boundary_scope.org.id
}

resource "boundary_account_password" "myuser" {
  name           = "admin"
  description    = "User account for adminr"
  login_name     = var.boundary_user
  password       = var.boundary_password
  auth_method_id = boundary_auth_method.password.id
}

resource "boundary_user" "myuser" {
  name        = "admin"
  description = "Initial Admin User"
  account_ids = [boundary_account_password.myuser.id]
  scope_id    = boundary_scope.org.id
}

resource "boundary_role" "org_admin" {
  scope_id        = "global"
  grant_scope_ids = [boundary_scope.org.id]
  grant_strings = [
    "ids=*;type=*;actions=*"
  ]
  principal_ids = [boundary_user.myuser.id]
}

resource "boundary_role" "project_admin" {
  scope_id        = boundary_scope.org.id
  grant_scope_ids = [boundary_scope.project.id]
  grant_strings = [
    "ids=*;type=*;actions=*"
  ]
  principal_ids = [boundary_user.myuser.id]

}

EOF

cat << EOF > ./init_boundary/variables.tf 
variable "boundary_user" {
  description = "Initial Boundary Admin User"
  type        = string
}

variable "boundary_password" {
  description = "Password for Boundary Admin"
  type        = string
}
EOF

cat << EOF > ./init_boundary/variables.tfvars
boundary_user        = "admin"            #Use your own
boundary_password    = "SuperSecret123!"  #Use your own
EOF

In [2]:
%%bash
cd init_boundary
terraform init
terraform apply --auto-approve --var-file=variables.tfvars


Initializing the backend...

Initializing provider plugins...
- Finding latest version of hashicorp/boundary...
- Installing hashicorp/boundary v1.1.15...
- Installed hashicorp/boundary v1.1.15 (signed by HashiCorp)

Terraform has created a lock file .terraform.lock.hcl to record the provider
selections it made above. Include this file in your version control repository
so that Terraform can guarantee to make the same selections by default when
you run "terraform init" in the future.

Terraform has been successfully initialized!

You may now begin working with Terraform. Try running "terraform plan" to see
any changes that are required for your infrastructure. All Terraform commands
should now work.

If you ever set or change modules or backend configuration for Terraform,
rerun this command to reinitialize your working directory. If you forget, other
commands will detect it and remind you to do so if necessary.

Terraform used the selected providers to generate the following executio

## To destroy configuration

In [3]:
%%bash
cd init_boundary
terraform destroy --auto-approve --var-file=variables.tfvars

boundary_scope.org: Refreshing state... [id=o_PltUFP0cRd]
boundary_scope.project: Refreshing state... [id=p_6zmi85vvx2]
boundary_auth_method.password: Refreshing state... [id=ampw_uzYmCCSOaU]
boundary_account_password.myuser: Refreshing state... [id=acctpw_nibbpEHPgd]
boundary_user.myuser: Refreshing state... [id=u_y1wXkLf5a2]
boundary_role.org_admin: Refreshing state... [id=r_cZOM893udg]
boundary_role.project_admin: Refreshing state... [id=r_IGTxyZ9TR6]

Terraform used the selected providers to generate the following execution
plan. Resource actions are indicated with the following symbols:
  - destroy

Terraform will perform the following actions:

  # boundary_account_password.myuser will be destroyed
  - resource "boundary_account_password" "myuser" {
      - auth_method_id = "ampw_uzYmCCSOaU" -> null
      - description    = "User account for adminr" -> null
      - id             = "acctpw_nibbpEHPgd" -> null
      - login_name     = "admin" -> null
      - name           = "admi